In [1]:
# XGBoost resnet featuremappide peal
# Osa koodi on Joosepi oma, uuendatud on XGBClassifier ja resnet osa

import pandas as pd
import numpy as np
import os
import PIL
from sklearn.multioutput import MultiOutputClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import f1_score
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50,  preprocess_input
from tensorflow.keras.preprocessing import image
from utils import load_train_images

In [2]:
def load_test_images():
    df = pd.read_csv('test.csv')
    imgs = []
    ids = []

    for img_id in df.image_id.values:
        
        try:
            
            image = tf.keras.preprocessing.image.load_img(os.path.join('images', img_id),
                    target_size=(300, 300),
                    keep_aspect_ratio = True,
                    color_mode = 'rgb')
            input_arr = tf.keras.preprocessing.image.img_to_array(image)
            
            image = np.expand_dims(input_arr, axis=0)
            image = image/255
            image = np.squeeze(image)

            imgs.append(image)
            ids.append(img_id)

        except FileNotFoundError:
            print(img_id, 'doesnt exist')

    return np.array(imgs), ids

In [3]:
def labelstring(onehot: np.ndarray) -> str:
    labels = np.array([f'l{i}' for i in range(92)])
    return ' '.join(labels[onehot[0]])

In [4]:
train_imgs, y_train = load_train_images()

model_resnet = ResNet50(weights='imagenet', include_top=False)

X_train = []
for img in train_imgs:
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    conv_featuremap = model_resnet.predict(x)
    fmap = conv_featuremap.mean(axis=(0,1,2))
    X_train.append(fmap)


img286.jpg doesnt exist
img172.jpg doesnt exist
img244.jpg doesnt exist
img287.jpg doesnt exist
1/1 [==============================] - 0s 485ms/step


In [5]:
xgboost = MultiOutputClassifier(XGBClassifier(scale_pos_weight=200, max_delta_step=5))
xgboost.fit(X_train, y_train)

MultiOutputClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              gpu_id=None, grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=5, max_depth=None,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=100, n_jobs=None,
                                              num_parallel_tree=None,
                                              predictor=None, random_state=None, ...))

In [6]:
prediction_on_train = xgboost.predict(X_train)

In [7]:
print(f'f1 score: {f1_score(y_train, prediction_on_train, average="macro"):.3f}')

f1 score: 1.000


In [8]:
test_imgs, test_img_ids = load_test_images()

img285.jpg doesnt exist
img288.jpg doesnt exist


In [9]:
X_test = []
for img in test_imgs:
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    conv_featuremap = model_resnet.predict(x)
    fmap = conv_featuremap.mean(axis=(0,1,2))
    X_test.append(fmap)


1/1 [==============================] - 0s 264ms/step


In [10]:
testdf = pd.read_csv('test.csv')

In [11]:
labelsdf = pd.read_csv('labels.csv')
testlabels = []
counter = 0

for img_id in testdf['image_id']:
    if img_id in test_img_ids:
        prediction = xgboost.predict(X_test[counter].reshape(1,-1))
        predicted_labels = labelstring(prediction.astype(bool))
        if len(predicted_labels) == 0:
            testlabels.append('l1')
        else:
            testlabels.append(predicted_labels)
        print('='*40)
        print(img_id)
        print(labelsdf.loc[labelsdf.label_id.isin(testlabels[-1].split(' ')), 'object'])
        print('='*40)
        counter += 1
    else:
        # default label for the missing images in our test set
        print('test image', img_id, 'missing from images')
        testlabels.append('l0')


testdf['labels'] = testlabels
testdf.to_csv('kea_submissions/resnet_xgboost_classifier2.csv', index=False)

img102.jpg
0    people
Name: object, dtype: object
img103.jpg
0    people
1     trees
2     grass
9      cars
Name: object, dtype: object
img11.jpg
0                                people
28    the sculpture of kissing students
Name: object, dtype: object
img113.jpg
1       trees
3    building
6       water
Name: object, dtype: object
img114.jpg
42    tractor
83      table
86      tents
Name: object, dtype: object
img121.jpg
0         people
1          trees
2          grass
3       building
6          water
10        nature
31    playground
Name: object, dtype: object
img126.jpg
0       people
1        trees
2        grass
3     building
6        water
7         road
9         cars
18      stairs
90        flag
Name: object, dtype: object
img131.jpg
0       people
2        grass
5    buildings
Name: object, dtype: object
img136.jpg
4        snow
44    glasses
66      chair
Name: object, dtype: object
img137.jpg
1          trees
3       building
21    stone road
73         house
Name: 